## 🍪  [Day 21](https://adventofcode.com/2018/day/21)

And here is another `ElfCode` problem.  the goal is to find which value of `R0` leads to the shortest program (that terminates). 

First we can get rid of the first code line, that only checks that `bani` acts on number inputs.

```
#ip 2
seti 123 0 5 --> Set R5 to 123
bani 5 456 5 --> R5 = 123 & 456
eqri 5 72 5 --> Set R5 to bool(R5 == 72)
addr 5 2 2 --> R2 += R5
seti 0 0 2
```
This code is equivalent to checking`123 & 456 == 72 ?`. If inputs are treated as numbers, this should be `True` and the instruction pointer skips to the actual program. Otherwise it loops back infinitely. Hence we can safely skip this code.


In [0]:
#@title Define all operations (see Day 16 and 19)
"""Defines all instructions"""
def apply_add(a, b, c, before, register=0):
  before[c] = before[a] + (before[b] if register else b)
  return before


def apply_mul(a, b, c, before, register=0):
  before[c] = before[a] * (before[b] if register else b)
  return before


def apply_ban(a, b, c, before, register=0):
  before[c] = before[a] & (before[b] if register else b)
  return before


def apply_bor(a, b, c, before, register=0):
  before[c] = before[a] | (before[b] if register else b)
  return before


def apply_set(a, b, c, before, register=0):
  before[c] = before[a] if register else a
  return before


def apply_gt(a, b, c, before, register=0):
  if register == 0:
    before[c] = a > before[b]
  elif register == 1:
    before[c] = before[a] > b
  else:
    before[c] = before[a] > before[b]    
  return before


def apply_eq(a, b, c, before, register=0):
  if register == 0:
    before[c] = a == before[b]
  elif register == 1:
    before[c] = before[a] == b
  else:
    before[c] = before[a] == before[b]    
  return before


"""Gather all instructions"""
from functools import partial
operations = {'addr': partial(apply_add, register=1),
              'addi': partial(apply_add, register=0),
              'mulr': partial(apply_mul, register=1), 
              'muli': partial(apply_mul, register=0),
              'banr': partial(apply_ban, register=1), 
              'bani': partial(apply_ban, register=0),
              'borr': partial(apply_bor, register=1), 
              'bori': partial(apply_bor, register=0),
              'setr': partial(apply_set, register=1), 
              'seti': partial(apply_set, register=0),
              'gtir': partial(apply_gt, register=0), 
              'gtri': partial(apply_gt, register=1),
              'gtrr': partial(apply_gt, register=2),
              'eqir': partial(apply_eq, register=0), 
              'eqri': partial(apply_eq, register=1),
              'eqrr': partial(apply_eq, register=2)}

In [0]:
def run_programm(inputs, init_registers=None, stop_on_instruction=None, verbose=False):
  """Run the background program"""
  registers = [0] * 6
  if init_registers is not None:
    registers = init_registers
  inst_register = inputs[0][1]
  instructions = inputs[1:]
  step = 0
  while 1:
    # run op
    op, a, b, c = instructions[registers[inst_register]]
    # stop if found instructions
    if stop_on_instruction is not None and registers[inst_register] == stop_on_instruction:
      print('Step %d' % (step + 1), op, a, b, c, '-->', registers)      
      break
    operations[op](a, b, c, registers)
    # stop if finished
    if registers[inst_register] >= len(instructions) - 1:
      break
    # next step
    registers[inst_register] += 1
    step += 1
    # verbose
    if verbose:
      print('Step %d' % (step + 1), op, a, b, c, '-->', registers)
  return step, registers

Once this test is done,  the program starts directly from instruction 5. Turns out the only instructions that refers to `R0` are the following:

```
24. eqrr 5 0 1 --> R1 = bool(R5 == R0)
25. addr 1 2 2  
26. seti 5 2 2
```
which translates as: if `R0` is equal to `R5` at line 24, then we exit the program; otherwise the program loops back from the start. In other words, to get the shortest programm, we need to determine when the program **first** arrives at `line 24`, (starting from zero registers); we know we can exit the program simply by setting `R0` to the value of `R5` at that point.

In [5]:
with open("day21.txt", 'r') as f:  
  inputs = f.read().splitlines()
  for i, line in enumerate(inputs):
    if line.startswith('#'):
      inputs[i] = ('inst', int(line[-1]), 0, 0)
    else:
      aux = line.split()
      inputs[i] = (aux[0], int(aux[1]), int(aux[2]), int(aux[3]))
      
r5_value = run_programm(inputs, stop_on_instruction=28)[1][5]
print('Setting R0 to %d, program finishes in %d instructions' % (
    r5_value, run_programm(inputs, init_registers=[r5_value, 0, 0, 0, 0, 0])[0]))            

Step 1847 eqrr 5 0 1 --> [0, True, 28, 1, True, 6619857]
Setting R0 to 6619857, program finishes in 1847 instructions


Now we need to get the value of `R0` for the longest execution that **still terminates**. In other words we need to find the highest value `R5` can achieve without ending up in a infinite loop. The core program is as follows

```
01. seti 0 3 5 
02. bori 5 65536 3
03. seti 9010242 6 5    
04. bani 3 255 1         
05. addr 5 1 5           
06. bani 5 16777215 5    
07. muli 5 65899 5       
08. bani 5 16777215 5    
09. gtir 256 3 1        
10. addr 1 2 2           
11. addi 2 1 2           
12. seti 27 6 2      
13. seti 0 8 1
14. addi 1 1 4
15. muli 4 256 4
16. gtrr 4 3 4
17. addr 4 2 2
18. addi 2 1 2
19. seti 25 5 2
20. addi 1 1 1
21. seti 17 7 2 
22. setr 1 3 3
23. seti 7 2 2 
```


In python pseudo code, this translates as:
```python
r5 = 0
while r0 != r5:
  r3 = 65536 | r5
  while r3 >= 256: 
    # instructions [9 - 15]
    r1 = r3 & 255
    r5 = (((9010242 + r1) & 16777215) * 65899) & 16777215    
    # [instructions 19 - 27]
    r3 = ceil(r3 // 256)     
```

When executing the program, we notice the values `R5` takes are cyclic, causing the program to fall into a loop. Consequently we need to set `R0`t o the last value reached by `R5` in that cycle to get the longest terminating run.


In [6]:
from math import ceil

history_r5 = []
r5 = 0
while r5 not in history_r5:
  history_r5.append(r5)
  r3 = 65536 | r5
  r5 = 9010242
  while 1:
    r1 = r3 & 255
    r5 = (((r5 + r1) & 16777215) * 65899) & 16777215
    if r3 >= 256:
      r3 = ceil(r3 // 256)
    else:
      break
r5_value = history_r5[-1]
print('R0 value for longest terminating run:', r5_value)   

R0 value for longest terminating run: 9547924


Just for fun, we can check how long running this program actually takes :)

In [7]:
%%time
print('Setting R0 to %d, program finishes in %d instructions' % (
    r5_value, run_programm(inputs, init_registers=[r5_value, 0, 0, 0, 0, 0])[0]))  

Setting R0 to 9547924, program finishes in 2346341652 instructions
CPU times: user 34min 14s, sys: 226 ms, total: 34min 14s
Wall time: 34min 14s
